### To-Do List

- [ ] Read data from repo
- [ ] split data with different seeds
- [ ] save each train and test for each seed in different folder





In [1]:
from ucimlrepo import fetch_ucirepo
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, LongType
import os
from utils.schemas import raw_data_schema

In [2]:
#train_path = os.environ.get("TRAIN_PATH")
#test_path = os.environ.get("TEST_PATH")
spark_master = os.environ.get("SPARK_MASTER_URL")
hadoop_master = os.environ.get("HADOOP_MASTER_URL")

spark = SparkSession.builder \
    .appName("MySparkApp") \
    .master(spark_master) \
    .config("spark.hadoop.fs.defaultFS", hadoop_master) \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/04 06:37:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
default_of_credit_card_clients = fetch_ucirepo(id=350) 
X = spark.createDataFrame(default_of_credit_card_clients.data.features)
y = spark.createDataFrame(default_of_credit_card_clients.data.targets)

# Print initial data info
print(f"Features shape: {X.count()} rows")
print(f"Targets shape: {y.count()} rows")

# combine X and y
X = X.withColumn("id", monotonically_increasing_id())
y = y.withColumn("id", monotonically_increasing_id())
df = X.join(y, on="id", how="inner").drop("id")

total_count = df.count()
print(f"Combined dataset size: {total_count} rows")

train, test = df.randomSplit([.7,.3], seed=42)


Features shape: 30000 rows
Targets shape: 30000 rows


Combined dataset size: 30000 rows


In [6]:
#df.write.mode('overwrite').options(header='True', delimiter=',').parquet('/mnt/train')
#df.write.mode('overwrite').options(header='True', delimiter=',').csv('/mnt/train.csv')
#test.write.options(header='True', delimiter=',').csv(test_path)
train.write.mode("overwrite").parquet("data/train")


In [10]:
train.coalesce(1).write.mode("overwrite") \
        #.option("schema", raw_data_schema.json()) \
        .parquet("data/train")

In [3]:
train_read = spark.read.schema(raw_data_schema).parquet('data/train')
#test_read = spark.read.schema(raw_data_schema).csv(test_path)

In [4]:
train_read.count()

21063

In [9]:
spark.stop()

In [12]:
print(pyarrow.__version__)

NameError: name 'pyarrow' is not defined

In [32]:
%pip install pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 3.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
write_data(train_path, test_path, spark)

Writing to absolute paths:
Train: /app/data/train
Test: /app/data/test


Features shape: 30000 rows


Targets shape: 30000 rows


Combined dataset size: 30000 rows


Split sizes - Train: 21063, Test: 8937
root
 |-- X1: long (nullable = true)
 |-- X2: long (nullable = true)
 |-- X3: long (nullable = true)
 |-- X4: long (nullable = true)
 |-- X5: long (nullable = true)
 |-- X6: long (nullable = true)
 |-- X7: long (nullable = true)
 |-- X8: long (nullable = true)
 |-- X9: long (nullable = true)
 |-- X10: long (nullable = true)
 |-- X11: long (nullable = true)
 |-- X12: long (nullable = true)
 |-- X13: long (nullable = true)
 |-- X14: long (nullable = true)
 |-- X15: long (nullable = true)
 |-- X16: long (nullable = true)
 |-- X17: long (nullable = true)
 |-- X18: long (nullable = true)
 |-- X19: long (nullable = true)
 |-- X20: long (nullable = true)
 |-- X21: long (nullable = true)
 |-- X22: long (nullable = true)
 |-- X23: long (nullable = true)
 |-- Y: long (nullable = true)

root
 |-- X1: long (nullable = true)
 |-- X2: long (nullable = true)
 |-- X3: long (nullable = true)
 |-- X4: long (nullable = true)
 |-- X5: long (nullable = true)
 |-- X6: 

Files successfully written to disk
Train directory exists: True
Test directory exists: True


In [8]:
train, test = read_data(train_path, test_path, spark)

Attempting to read from:
Train: /app/data/train
Test: /app/data/test
Train directory contents: ['._SUCCESS.crc', '_SUCCESS']
Test directory contents: ['._SUCCESS.crc', '_SUCCESS']
Schema of loaded train data:
root
 |-- X1: long (nullable = true)
 |-- X2: long (nullable = true)
 |-- X3: long (nullable = true)
 |-- X4: long (nullable = true)
 |-- X5: long (nullable = true)
 |-- X6: long (nullable = true)
 |-- X7: long (nullable = true)
 |-- X8: long (nullable = true)
 |-- X9: long (nullable = true)
 |-- X10: long (nullable = true)
 |-- X11: long (nullable = true)
 |-- X12: long (nullable = true)
 |-- X13: long (nullable = true)
 |-- X14: long (nullable = true)
 |-- X15: long (nullable = true)
 |-- X16: long (nullable = true)
 |-- X17: long (nullable = true)
 |-- X18: long (nullable = true)
 |-- X19: long (nullable = true)
 |-- X20: long (nullable = true)
 |-- X21: long (nullable = true)
 |-- X22: long (nullable = true)
 |-- X23: long (nullable = true)
 |-- Y: long (nullable = true)

Init

ValueError: Read data is empty

In [4]:
def write_data(train_path, test_path, spark):
    # Ensure absolute paths
    train_path = os.path.abspath(train_path)
    test_path = os.path.abspath(test_path)
    
    print(f"Writing to absolute paths:\nTrain: {train_path}\nTest: {test_path}")
    
    # fetch dataset 
    default_of_credit_card_clients = fetch_ucirepo(id=350) 
    X = spark.createDataFrame(default_of_credit_card_clients.data.features)
    y = spark.createDataFrame(default_of_credit_card_clients.data.targets)

    # Print initial data info
    print(f"Features shape: {X.count()} rows")
    print(f"Targets shape: {y.count()} rows")

    # combine X and y
    X = X.withColumn("id", monotonically_increasing_id())
    y = y.withColumn("id", monotonically_increasing_id())
    df = X.join(y, on="id", how="inner").drop("id")
    
    total_count = df.count()
    print(f"Combined dataset size: {total_count} rows")

    train, test = df.randomSplit([.7,.3], seed=42)
    
    # Get counts before writing
    train_count = train.count()
    test_count = test.count()
    print(f"Split sizes - Train: {train_count}, Test: {test_count}")

    if train_count == 0 or test_count == 0:
        raise ValueError("Train or test dataset is empty after split")
        
    # Ensure the directory exists
    os.makedirs(os.path.dirname(train_path), exist_ok=True)
    os.makedirs(os.path.dirname(test_path), exist_ok=True)

    train.printSchema()
    test.printSchema()
    
    # Write with coalesce instead of repartition for smaller files
    train.coalesce(1).write.mode("overwrite").parquet(train_path)
    test.coalesce(1).write.mode("overwrite").parquet(test_path)
    
    # Verify files exist after writing
    if not os.path.exists(train_path) or not os.path.exists(test_path):
        raise FileNotFoundError("Failed to write parquet files")
        
    print(f"Files successfully written to disk")
    print(f"Train directory exists: {os.path.exists(train_path)}")
    print(f"Test directory exists: {os.path.exists(test_path)}")

In [7]:
def read_data(train_path, test_path, spark):
    print(f"Attempting to read from:\nTrain: {train_path}\nTest: {test_path}")
    
    if not os.path.exists(train_path):
        raise FileNotFoundError(f"Train path does not exist: {train_path}")
    if not os.path.exists(test_path):
        raise FileNotFoundError(f"Test path does not exist: {test_path}")
        
    # List contents of directories
    print(f"Train directory contents: {os.listdir(train_path)}")
    print(f"Test directory contents: {os.listdir(test_path)}")
    
    try:
        # Read with predefined schema
        train = spark.read.schema(raw_data_schema).parquet(train_path)
        test = spark.read.schema(raw_data_schema).parquet(test_path)
        
        print("Schema of loaded train data:")
        train.printSchema()
        
        # Verify the data was read successfully
        train_count = train.count()
        test_count = test.count()
        
        print(f"Initial read - Train count: {train_count}, Test count: {test_count}")
        
        if train_count == 0 or test_count == 0:
            raise ValueError("Read data is empty")
            
        print(f"Train data read: {train_count} rows")
        print(f"Test data read: {test_count} rows")
        
        return train, test
    except Exception as e:
        print(f"Error reading parquet files: {str(e)}")
        raise
